## Manifest and Metadata Creation

In [2]:
import pandas as pd 
import numpy as np

In [44]:
control = pd.read_csv('control_fastq.csv',index_col=0, dtype={'sample_names': str, 'biosample_ids': str}).reset_index(drop=True)
treatment = pd.read_csv('treatment_fastq.csv',index_col=0, dtype={'sample_names': str, 'biosample_ids': str}).reset_index(drop=True)
data_id = pd.concat([control, treatment]).reset_index(drop=True)

In [45]:
control.describe()

,sample_names,biosample_ids,run_ids
count,120,120,120
unique,120,120,116
top,10317.000001001,4186901,Not found
freq,1,1,5


In [46]:
data_id

,sample_names,biosample_ids,run_ids
0,10317.000001001,4186901,ERR1073439
1,10317.000001002,4186120,ERR1072624
2,10317.000001004,4186121,ERR1072625
3,10317.000001008,4186122,ERR1072626
4,10317.000001018,4186123,ERR1089664
...,...,...,...
225,10317.000109807,18046629,ERR5332367
226,10317.000110177,18046932,ERR5329115
227,10317.000112912,18046822,ERR5333108
228,10317.000113088,18045873,ERR5332725


In [47]:
manifest = data_id.sample_names.to_frame()

In [52]:
manifest = manifest.rename(columns={'sample_names': 'sample-id'})

In [40]:
manifest

,sample_names
0,10317.000001001
1,10317.000001002
2,10317.000001004
3,10317.000001008
4,10317.000001018
...,...
105,10317.000109807
106,10317.000110177
107,10317.000112912
108,10317.000113088


In [54]:
# Costruiamo e teniamo solo le colonne che ci servono
manifest['absolute-filepath'] = '/home/LABORATORI/gr887277/gut-microbiota-diet/good_quality_20'+ data_id['run_ids']
manifest['direction'] = 'forward'

manifest = manifest[['sample-id', 'absolute-filepath', 'direction']]

# Scriviamo il csv
manifest.to_csv('fastq_manifest.csv', sep=',', index=False)
manifest

,sample-id,absolute-filepath,direction
0,10317.000001001,/home/LABORATORI/gr887277/gut-microbiota-diet/...,forward
1,10317.000001002,/home/LABORATORI/gr887277/gut-microbiota-diet/...,forward
2,10317.000001004,/home/LABORATORI/gr887277/gut-microbiota-diet/...,forward
3,10317.000001008,/home/LABORATORI/gr887277/gut-microbiota-diet/...,forward
4,10317.000001018,/home/LABORATORI/gr887277/gut-microbiota-diet/...,forward
...,...,...,...
225,10317.000109807,/home/LABORATORI/gr887277/gut-microbiota-diet/...,forward
226,10317.000110177,/home/LABORATORI/gr887277/gut-microbiota-diet/...,forward
227,10317.000112912,/home/LABORATORI/gr887277/gut-microbiota-diet/...,forward
228,10317.000113088,/home/LABORATORI/gr887277/gut-microbiota-diet/...,forward


## Metadata file creation

In [44]:
data = pd.read_csv('ref_sugar_sample.csv')

sample_name,age_years,age_cat,sex,specialized_diet_exclude_dairy,specialized_diet_exclude_refined_sugars,body_site,specialized_diet_i_do_not_eat_a_specialized_diet,specialized_diet_paleodiet_or_primal_diet

In [46]:
# Aggiungiamo al manifest le informazioni riguardo l'età, il sesso e l'uso degli antibiotici
df_metadata = pd.merge(manifest['sample-id'], 
                       data[['sample_name', 'sex', 'age_cat', 'specialized_diet_exclude_refined_sugars']], 
                       how='inner', 
                       left_on='sample-id', 
                       right_on = 'sample_name')

df_metadata = df_metadata.drop(['sample_name'], axis=1)

import numpy as np

#conditions = [
#    (df_metadata['antibiotic_history']=='Year'),
#    (df_metadata['antibiotic_history']=='6 months'),
#    (df_metadata['antibiotic_history']=='Month'),
#    (df_metadata['antibiotic_history']=='Week'),
#]

#choices = [
#    'B - Year',
#    'C - 6 months',
#    'D - Month',
#    'E - Week']

#df_metadata['antibiotic_history_ord'] = np.select(conditions, choices, default=' A - I have not taken antibiotics in the past year.')
               
# Scrviamo il tsv
df_metadata.to_csv('sample_metadata.tsv', sep='\t', index=False)
df_metadata

,sample-id,sex,age_cat,specialized_diet_exclude_refined_sugars
0,10317.000023,female,60s,Yes
1,10317.000024,male,50s,Yes
2,10317.000030,male,20s,Yes
3,10317.000033,female,60s,Yes
4,10317.000035,female,40s,Yes
...,...,...,...,...
262,10317.000110,male,60s,Yes
263,10317.000110,male,70+,Yes
264,10317.000110,male,60s,Yes
265,10317.000110,male,70+,Yes


In [1]:
%conda update conda
%conda install wget

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::nbclassic==0.5.2=py39h06a4308_0
  - defaults/linux-64::ipykernel==6.19.2=py39hb070fc8_0
  - defaults/linux-64::jupyter==1.0.0=py39h06a4308_8
  - defaults/linux-64::jupyterlab==3.5.3=py39h06a4308_0
  - defaults/linux-64::spyder==5.3.3=py39h06a4308_0
  - defaults/linux-64::_ipyw_jlab_nb_ext_conf==0.1.0=py39h06a4308_1
  - defaults/noarch::jupyterlab_server==2.10.3=pyhd3eb1b0_1
  - defaults/noarch::jupyterlab_pygments==0.1.2=py_0
  - defaults/linux-64::holoviews==1.15.4=py39h06a4308_0
  - defaults/linux-64::scikit-learn-intelex==2021.6.0=py39h06a4308_0
  - defaults/linux-64::jupyter_server==1.23.4=py39h06a4308_0
  - defaults/linux-64::_anaconda_depends==2022.10=py39_2
  - defaults/linux-64::widgetsnbextension==3.5.2=py39h06a4308_0
  - defaults/linux-64::notebook==6.5.2=py39h06a4308_0
  - defaults/linux-64::anaconda==c

In [3]:
!wget https://data.qiime2.org/distro/core/qiime2-2023.2-py38-linux-conda.yml
%conda env create -n qiime2-2023.2 --file qiime2-2023.2-py38-linux-conda.yml
%conda activate qiime2-2023.2

--2023-05-10 15:47:17--  https://data.qiime2.org/distro/core/qiime2-2023.2-py38-linux-conda.yml
Resolving data.qiime2.org (data.qiime2.org)... 54.200.1.12
Connecting to data.qiime2.org (data.qiime2.org)|54.200.1.12|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://raw.githubusercontent.com/qiime2/environment-files/master/2023.2/release/qiime2-2023.2-py38-linux-conda.yml [following]
--2023-05-10 15:47:17--  https://raw.githubusercontent.com/qiime2/environment-files/master/2023.2/release/qiime2-2023.2-py38-linux-conda.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14704 (14K) [text/plain]
Saving to: ‘qiime2-2023.2-py38-linux-conda.yml’

qiime2-2023.2-py38- 100%[===================>]  14,36K  --.-KB/s    in 0s      

20

In [1]:
!qiime --help

Usage: qiime [OPTIONS] COMMAND [ARGS]...

  QIIME 2 command-line interface (q2cli)
  --------------------------------------

  To get help with QIIME 2, visit https://qiime2.org.

  To enable tab completion in Bash, run the following command or add it to
  your .bashrc/.bash_profile:

      source tab-qiime

  To enable tab completion in ZSH, run the following commands or add them to
  your .zshrc:

      autoload -Uz compinit && compinit
      autoload bashcompinit && bashcompinit
      source tab-qiime

Options:
  --version   Show the version and exit.
  --help      Show this message and exit.

Commands:
  info                Display information about current deployment.
  tools               Tools for working with QIIME 2 files.
  dev                 Utilities for developers and advanced users.
  alignment           Plugin for generating and manipulating alignments.
  composition         Plugin for compositional data analysis.
  cutadapt            Plugin for removing adapter sequen

In [2]:
from qiime2 import Artifact
artifact_data = Artifact.import_data('SampleData[SequencesWithQuality]',
                                     '/c/Users/adm/Desktop/Bioscience/gut-microbiota-diet/fastq_manifest.csv',
                                     view_type='SingleEndFastqManifestPhred33')

artifact_data.save('artifact_data.qza')

ValidationError: Path '/c/Users/adm/Desktop/Bioscience/gut-microbiota-diet/fastq_manifest.csv' does not exist.

In [ ]:
artifact_data = Artifact.load('artifact_data.qza')
artifact_data
